In [1]:
!pip install bs4
from bs4 import BeautifulSoup
!pip install selenium
import random
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd
import time

In [2]:
df_links=pd.read_excel("ConvocatoriasPublicas.xlsx")

df_links["union"]=df_links['CodigoConvocatoria']+"/"+df_links['NombreAgenteComprador']+"/"+df_links['CodigoSicAgente']+"/01%2F02%2F2023/31%2F12%2F2026"
df_links

,CodigoConvocatoria,CodigoSicAgente,NombreAgenteComprador,FechaPublicacionAviso,PeriodoAContratar,FechaPliegosParaConsulta,FechaPliegosDefinitivos,FechaLimiteRecepcionOfertas,AudienciaPublica,FechaMaximaFormalizacionContratos,Estado,FechaSuspension,FechaReactivacion,FechaCancelacion,FechaCierreConvocatoria,union
0,CP-EMPC2022-002,EMPC,EMSERPUCAR,02/12/2022,01/02/2023 al 31/12/2026,02/12/2022,14/12/2022,12/01/2023,25/01/2023,15/02/2023,Cerrada y adjudicada,NaN,NaN,NaN,16/02/2023,CP-EMPC2022-002/EMSERPUCAR/EMPC/01%2F02%2F2023...
1,CP-GNCC2022-003,GNCC,VATIA,17/11/2022,01/02/2023 al 31/12/2026,21/11/2022,07/12/2022,05/01/2023,19/01/2023,27/01/2023,Cerrada y adjudicada,NaN,NaN,NaN,31/01/2023,CP-GNCC2022-003/VATIA/GNCC/01%2F02%2F2023/31%2...
2,CP-TENC2022-003,TENC,Tenergeticas,09/11/2022,01/02/2023 al 31/12/2033,10/11/2022,21/11/2022,20/12/2022,30/12/2022,23/01/2023,Cerrada y adjudicada,NaN,NaN,NaN,14/02/2023,CP-TENC2022-003/Tenergeticas/TENC/01%2F02%2F20...
3,CP-BIAC2022-002,BIAC,BIAE,01/11/2022,01/02/2023 al 31/12/2038,01/11/2022,10/11/2022,12/12/2022,23/12/2022,13/01/2023,Cerrada y adjudicada,NaN,NaN,NaN,17/01/2023,CP-BIAC2022-002/BIAE/BIAC/01%2F02%2F2023/31%2F...
4,CP-EEPC2022-003,EEPC,EEP,26/10/2022,01/01/2023 al 31/12/2023,26/10/2022,08/11/2022,07/12/2022,21/12/2022,30/12/2022,Cerrada y adjudicada,NaN,NaN,NaN,02/01/2023,CP-EEPC2022-003/EEP/EEPC/01%2F02%2F2023/31%2F1...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,CP-PEEC2020-002,PEEC,PEESA,03/03/2020,01/06/2020 al 31/12/2022,04/03/2020,26/03/2020,27/04/2020,09/05/2020,29/05/2020,Cerrada y adjudicada,NaN,NaN,NaN,03/06/2020,CP-PEEC2020-002/PEESA/PEEC/01%2F02%2F2023/31%2...
148,CP-GNCC2020-001,GNCC,VATIA S.A.,20/02/2020,01/08/2020 al 31/12/2025,20/02/2020,12/03/2020,29/04/2020,13/05/2020,01/06/2020,Cerrada y adjudicada,14/04/2020,NaN,NaN,04/06/2020,CP-GNCC2020-001/VATIA S.A./GNCC/01%2F02%2F2023...
149,CP-EDIC2020-001,EDIC,ELECTRICARIBE.,11/02/2020,15/05/2020 al 31/12/2025,11/02/2020,02/03/2020,31/03/2020,07/05/2020,29/05/2020,Cerrada y adjudicada,13/04/2020,NaN,NaN,01/06/2020,CP-EDIC2020-001/ELECTRICARIBE./EDIC/01%2F02%2F...
150,CP-EPIC2020-001,EPIC,CELSIA TOLIMA,03/02/2020,01/05/2020 al 31/12/2024,03/02/2020,24/02/2020,24/03/2020,03/04/2020,27/04/2020,Cerrada y adjudicada,NaN,NaN,NaN,28/04/2020,CP-EPIC2020-001/CELSIA TOLIMA/EPIC/01%2F02%2F2...


In [47]:
def scraping(link):
    #Ubicacion de webdrive
    ubicacion = "D:\\Descargas\\chromedriver_win32\\chromedriver" #Ruta del driver
    driver = webdriver.Chrome(ubicacion)

    #Definimos link
    home_link = "https://sicep.xm.com.co/public-announcements/electronic-file/"
    search_kw = str(link)
    driver.get(home_link+search_kw)

    #abrimos pagina
    page = BeautifulSoup(driver.page_source,"html.parser")
    #tiempo de espera
    time.sleep(7)
    # Debemos darle click al boton de disclaimer para que no interrumpa nuestras acciones
    try: # Encerramos todo en un try catch para que si no aparece el discilamer, no se caiga el codigo
        disclaimer1 = driver.find_element(By.XPATH, '//*[@id="expediente"]/div/p-accordiontab[1]')
        disclaimer1.click() # lo obtenemos y le damos click
    except Exception as e:
        print (e) 
        None
    
    #tiempo de espera
    time.sleep(4)
    
    #buscamos fecha inicio
    periodo_inicio = driver.find_element('xpath', '//*[@id="titleperiodtohire"]/div[1]/p[2]')
    periodo_inicio = periodo_inicio.text.strip()

    #buscamos fecha final
    periodo_fin = driver.find_element('xpath', '//*[@id="titleperiodtohire"]/div[2]/p[2]')
    periodo_fin = periodo_fin.text.strip()

    #descripcion
    descripcion = driver.find_element('xpath', '//*[@id="detConvocatoria"]/div[5]/div/p')
    descripcion = descripcion.text.strip()

    #precios
    # Debemos darle click al boton de disclaimer para que no interrumpa nuestras acciones
    try: # Encerramos todo en un try catch para que si no aparece el discilamer, no se caiga el codigo
        disclaimer = driver.find_element(By.XPATH, '//*[@id="expediente"]/div/p-accordiontab[7]')
        disclaimer.click() # lo obtenemos y le damos click
    except Exception as e:
        print (e) 
    None
    # Esperar 2 segundos
    time.sleep(4)
    # Encontrar el elemento que contiene la tabla y obtener su contenido HTML
    tabla_elements = driver.find_elements('xpath','//*[@id="ui-accordiontab-6-content"]/div/app-publication-information/div/div[2]/div[3]/p-table/div/div')
    tabla_html_list = []
    for tabla_element in tabla_elements:
        tabla_html = tabla_element.get_attribute('innerHTML')
        tabla_html_list.append(tabla_html)
    # Utilizar BeautifulSoup para analizar el contenido HTML y extraer la información que necesitas
    soup = BeautifulSoup(tabla_html, 'html.parser')
    tabla = soup.find('table')
    # Imprimir la tabla
    tabla_html = tabla.prettify()

    # Convertir la tabla HTML en un DataFrame
    import pandas as pd
    df = pd.read_html(tabla_html)[0]
    df["periodo_inicio"]=periodo_inicio
    df["periodo_fin"]=periodo_fin
    df["descripcion"]=descripcion
    df["union"]=link
    time.sleep(1)
    driver.quit()
    return df

Creamos resultado

In [23]:
link=df_links.union[0]
df_resultado = scraping(link)
df_resultado = df_links.iloc[[0]].merge(df_resultado, on='union')

C:\Users\DELLPHOTO\AppData\Local\Temp\ipykernel_9852\1979202400.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ubicacion)


In [57]:
for i in range(147,152):
    print(i)
    link=df_links.union[i]
    df_scr = scraping(link)
    df_scr = df_links.iloc[[i]].merge(df_scr, on='union')

    #concadenamos al resultado
    df_resultado = pd.concat([df_resultado, df_scr], ignore_index=True)

    #tiempo de espera
    time.sleep(10)

    


147


C:\Users\DELLPHOTO\AppData\Local\Temp\ipykernel_9852\1199038660.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ubicacion)


148
149
150
151
152


KeyError: 152

In [54]:
#141 146 falla
df_links

,CodigoConvocatoria,CodigoSicAgente,NombreAgenteComprador,FechaPublicacionAviso,PeriodoAContratar,FechaPliegosParaConsulta,FechaPliegosDefinitivos,FechaLimiteRecepcionOfertas,AudienciaPublica,FechaMaximaFormalizacionContratos,Estado,FechaSuspension,FechaReactivacion,FechaCancelacion,FechaCierreConvocatoria,union,tmp
0,CP-EMPC2022-002,EMPC,EMSERPUCAR,02/12/2022,01/02/2023 al 31/12/2026,02/12/2022,14/12/2022,12/01/2023,25/01/2023,15/02/2023,Cerrada y adjudicada,NaN,NaN,NaN,16/02/2023,CP-EMPC2022-002/EMSERPUCAR/EMPC/01%2F02%2F2023...,1
1,CP-GNCC2022-003,GNCC,VATIA,17/11/2022,01/02/2023 al 31/12/2026,21/11/2022,07/12/2022,05/01/2023,19/01/2023,27/01/2023,Cerrada y adjudicada,NaN,NaN,NaN,31/01/2023,CP-GNCC2022-003/VATIA/GNCC/01%2F02%2F2023/31%2...,1
2,CP-TENC2022-003,TENC,Tenergeticas,09/11/2022,01/02/2023 al 31/12/2033,10/11/2022,21/11/2022,20/12/2022,30/12/2022,23/01/2023,Cerrada y adjudicada,NaN,NaN,NaN,14/02/2023,CP-TENC2022-003/Tenergeticas/TENC/01%2F02%2F20...,1
3,CP-BIAC2022-002,BIAC,BIAE,01/11/2022,01/02/2023 al 31/12/2038,01/11/2022,10/11/2022,12/12/2022,23/12/2022,13/01/2023,Cerrada y adjudicada,NaN,NaN,NaN,17/01/2023,CP-BIAC2022-002/BIAE/BIAC/01%2F02%2F2023/31%2F...,1
4,CP-EEPC2022-003,EEPC,EEP,26/10/2022,01/01/2023 al 31/12/2023,26/10/2022,08/11/2022,07/12/2022,21/12/2022,30/12/2022,Cerrada y adjudicada,NaN,NaN,NaN,02/01/2023,CP-EEPC2022-003/EEP/EEPC/01%2F02%2F2023/31%2F1...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,CP-PEEC2020-002,PEEC,PEESA,03/03/2020,01/06/2020 al 31/12/2022,04/03/2020,26/03/2020,27/04/2020,09/05/2020,29/05/2020,Cerrada y adjudicada,NaN,NaN,NaN,03/06/2020,CP-PEEC2020-002/PEESA/PEEC/01%2F02%2F2023/31%2...,1
148,CP-GNCC2020-001,GNCC,VATIA S.A.,20/02/2020,01/08/2020 al 31/12/2025,20/02/2020,12/03/2020,29/04/2020,13/05/2020,01/06/2020,Cerrada y adjudicada,14/04/2020,NaN,NaN,04/06/2020,CP-GNCC2020-001/VATIA S.A./GNCC/01%2F02%2F2023...,1
149,CP-EDIC2020-001,EDIC,ELECTRICARIBE.,11/02/2020,15/05/2020 al 31/12/2025,11/02/2020,02/03/2020,31/03/2020,07/05/2020,29/05/2020,Cerrada y adjudicada,13/04/2020,NaN,NaN,01/06/2020,CP-EDIC2020-001/ELECTRICARIBE./EDIC/01%2F02%2F...,1
150,CP-EPIC2020-001,EPIC,CELSIA TOLIMA,03/02/2020,01/05/2020 al 31/12/2024,03/02/2020,24/02/2020,24/03/2020,03/04/2020,27/04/2020,Cerrada y adjudicada,NaN,NaN,NaN,28/04/2020,CP-EPIC2020-001/CELSIA TOLIMA/EPIC/01%2F02%2F2...,1


In [53]:
df_resultado.to_csv("final.csv")

In [115]:
ubicacion = "D:\\Descargas\\chromedriver_win32\\chromedriver" #Ruta del driver
driver = webdriver.Chrome(ubicacion)

C:\Users\DELLPHOTO\AppData\Local\Temp\ipykernel_5768\557961577.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ubicacion)


In [116]:
home_link = "https://sicep.xm.com.co/public-announcements/electronic-file/"
search_kw = str(df_links.union[1])

driver.get(home_link+search_kw)

page = BeautifulSoup(driver.page_source,"html.parser")

#bloque = page.find('li', attrs={'class':'s-item', 'data-view':True})

In [117]:
# Debemos darle click al boton de disclaimer para que no interrumpa nuestras acciones
try: # Encerramos todo en un try catch para que si no aparece el discilamer, no se caiga el codigo
  disclaimer1 = driver.find_element(By.XPATH, '//*[@id="expediente"]/div/p-accordiontab[1]')
  disclaimer1.click() # lo obtenemos y le damos click
except Exception as e:
  print (e) 
  None

In [118]:
periodo_inicio = driver.find_element('xpath', '//*[@id="titleperiodtohire"]/div[1]/p[2]')
periodo_inicio = periodo_inicio.text.strip()
periodo_inicio

'01/02/2023'

In [119]:
periodo_fin = driver.find_element('xpath', '//*[@id="titleperiodtohire"]/div[2]/p[2]')
periodo_fin = periodo_fin.text.strip()
periodo_fin

'31/12/2026'

In [120]:
descipcion = driver.find_element('xpath', '//*[@id="detConvocatoria"]/div[5]/div/p')
descipcion = descipcion.text.strip()
descipcion

'VATIA S.A E.S.P. -VATIA- está interesada en recibir ofertas para la contratación del suministro de energía eléctrica con destino a atender la demanda del mercado regulado en el periodo comprendido desde las 00:00 horas de la fecha de registro hasta las 24:00 del día 31 de diciembre de 2026.'

In [82]:
import time

In [121]:
# Debemos darle click al boton de disclaimer para que no interrumpa nuestras acciones
try: # Encerramos todo en un try catch para que si no aparece el discilamer, no se caiga el codigo
  disclaimer = driver.find_element(By.XPATH, '//*[@id="expediente"]/div/p-accordiontab[7]')
  disclaimer.click() # lo obtenemos y le damos click
except Exception as e:
  print (e) 
  None
# Esperar 2 segundos
time.sleep(2)
# Encontrar el elemento que contiene la tabla y obtener su contenido HTML
tabla_elements = driver.find_elements('xpath','//*[@id="ui-accordiontab-6-content"]/div/app-publication-information/div/div[2]/div[3]/p-table/div/div')
tabla_html_list = []
for tabla_element in tabla_elements:
    tabla_html = tabla_element.get_attribute('innerHTML')
    tabla_html_list.append(tabla_html)
# Utilizar BeautifulSoup para analizar el contenido HTML y extraer la información que necesitas
soup = BeautifulSoup(tabla_html, 'html.parser')
tabla = soup.find('table')
# Imprimir la tabla
tabla_html = tabla.prettify()

In [122]:
# Convertir la tabla HTML en un DataFrame
import pandas as pd
df = pd.read_html(tabla_html)[0]
df

,Productos,Cantidad de energía demandada GWh,Cantidad de energía adjudicada GWh,Precio promedio ponderado adjudicado ($/KWh)
0,CP-GNCC2022-003-PROD01,115.00,108.75,276.14
1,CP-GNCC2022-003-PROD02,235.60,235.60,281.90
2,CP-GNCC2022-003-PROD03,135.55,0.00,0.00
3,CP-GNCC2022-003-PROD04,161.42,0.00,0.00


In [92]:
driver.quit()

**DATOS AUTOGENERACION

In [93]:
!pip install pydataxm 
from pydataxm import *                           #Se realiza la importación de las librerias necesarias para ejecutar
import datetime as dt  
objetoAPI = pydataxm.ReadDB()   

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 0.0/52.1 kB ? eta -:--:--
     ---------------------------------------- 52.1/52.1 kB ? eta 0:00:00
  Created wheel for pydataxm: filename=pydataxm-0.3.2-py3-none-any.whl size=3493 sha256=d593edd42fb9a2d688577dd62cfb2e1914bddd9ffaeca1bf28fa9dc18964d9cc
  Stored in directory: c:\users\dellphoto\appdata\local\pip\cache\wheels\1c\c2\b2\f1c7c7d38ed36ef15e596aeef07aba673d7d9650f4134b2ce9
Successfully built pydataxm


In [105]:
datos_AGPE=objetoAPI.request_data(
                                'ListadoAGPE',
                                'Agente', 
                                dt.date(2001, 1, 1), 
                                dt.date(2002, 1, 31))
datos_AGPE

,Id,Values_AgpeID,Values_Name,Values_CompanyCode,Values_ComZone,Values_VoltageLevel,Values_EnerSource,Values_InstalledCapacity,Date
0,Agente,Agp00001,PROMOTORA RD LTDA,CMMC,CARIBE MAR,2,RAD SOLAR,9.000000,2023-03-27
1,Agente,Agp00002,ORGANIZACION TERPEL S A,EPSC,VALLE DEL CAUCA,2,GAS,16.011109,2023-03-27
2,Agente,Agp00003,ANDREA BARRIOS,CNSC,NORTE DE SANTANDER,1,RAD SOLAR,5.000000,2023-03-27
3,Agente,Agp00004,JUAN JOSE SANTA VALLA,CMMC,CARIBE MAR,2,RAD SOLAR,10.000000,2023-03-27
4,Agente,Agp00005,LUIS ALFONSO VILLAMIL VARGAS,CSSC,CARIBE SOL,1,RAD SOLAR,3.000000,2023-03-27
...,...,...,...,...,...,...,...,...,...
5140,Agente,Grp0598,AGPE INVERSIONES SANTA FE SEDE II,GNCC,CARIBE MAR,1,RAD SOLAR,15.000000,2023-03-27
5141,Agente,Grp0601,AGPE GATO DUMAS BOGOTA,BIAC,BOGOTA - CUNDINAMARCA,1,RAD SOLAR,44.800000,2023-03-27
5142,Agente,Unt0499,AGPE - ECOPETROL LA HORMIGA,GECG,BAJO PUTUMAYO,3,GAS,1000.000000,2023-03-27
5143,Agente,Unt0514,AGPE ENTREPALMAS,EPSG,META,2,BIOGAS,500.000000,2023-03-27


In [106]:
compra_AGPE=objetoAPI.request_data(
                                'ExcedenteAGPE',
                                'Agente', 
                                dt.date(2021, 1, 1), 
                                dt.date(2021, 1, 31))


,Id,Values_code,Values_Hour01,Values_Hour02,Values_Hour03,Values_Hour04,Values_Hour05,Values_Hour06,Values_Hour07,Values_Hour08,...,Values_Hour16,Values_Hour17,Values_Hour18,Values_Hour19,Values_Hour20,Values_Hour21,Values_Hour22,Values_Hour23,Values_Hour24,Date
0,Agente,CASC,NaN,NaN,NaN,NaN,NaN,NaN,3.24,26.47,...,6.74,0.32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-01-01
1,Agente,CEOC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-01-01
2,Agente,CETC,NaN,NaN,NaN,NaN,NaN,NaN,0.01,3.17,...,26.38,14.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-01-01
3,Agente,CHCC,0.01,NaN,NaN,NaN,NaN,NaN,3.14,33.55,...,64.75,25.06,4.03,0.02,0.03,0.05,0.04,0.05,0.05,2021-01-01
4,Agente,CMMC,NaN,NaN,NaN,0.00,NaN,NaN,1.25,57.51,...,137.50,51.09,2.01,0.00,0.01,NaN,NaN,NaN,NaN,2021-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
471,Agente,ENIC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-01-31
472,Agente,EPMC,3.08,5.29,5.45,3.78,3.81,4.11,5.71,41.58,...,1172.35,469.43,72.42,6.70,4.10,4.12,4.17,4.82,4.50,2021-01-31
473,Agente,ESSC,0.20,0.21,0.22,0.22,0.22,0.21,4.70,104.05,...,254.10,107.58,8.06,0.20,0.19,0.18,0.20,0.20,0.21,2021-01-31
474,Agente,HLAC,1.00,0.58,0.58,0.98,0.58,0.98,0.92,33.25,...,47.14,8.09,1.64,0.20,0.58,0.58,0.58,0.98,0.58,2021-01-31


In [111]:
datos_AGPE[datos_AGPE["Values_CompanyCode"]=="HLAC"]["Values_InstalledCapacity"].sum()

2339.425

In [109]:
compra_AGPE[compra_AGPE["Values_code"]=="HLAC"].mean()

C:\Users\DELLPHOTO\AppData\Local\Temp\ipykernel_5768\2271267194.py:1: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  compra_AGPE[compra_AGPE["Values_code"]=="HLAC"].mean()
C:\Users\DELLPHOTO\AppData\Local\Temp\ipykernel_5768\2271267194.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  compra_AGPE[compra_AGPE["Values_code"]=="HLAC"].mean()


Values_Hour01      0.598065
Values_Hour02      0.583226
Values_Hour03      0.583226
Values_Hour04      0.597419
Values_Hour05      0.586452
Values_Hour06      0.598065
Values_Hour07      0.785806
Values_Hour08     12.577097
Values_Hour09     47.088387
Values_Hour10     98.871613
Values_Hour11    150.102903
Values_Hour12    182.498710
Values_Hour13    210.304839
Values_Hour14    178.728710
Values_Hour15    108.965806
Values_Hour16     54.773871
Values_Hour17     15.119032
Values_Hour18      1.233000
Values_Hour19      0.303548
Values_Hour20      0.570968
Values_Hour21      0.576774
Values_Hour22      0.580645
Values_Hour23      0.600645
Values_Hour24      0.582581
dtype: float64